# Procesamiento de lenguaje natural
## Bot con NLTK utilizando un corpus de wikipedia


In [1]:
import json
import string
import random
import re
import urllib.request

import numpy as np
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
# Descargar el diccionario
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Datos
Se consumira los datos del artículo de wikipedia sobre el deporte "tennis" en ingles.

In [9]:
with open("/content/drive/MyDrive/Colab Notebooks/Gutenbergs The Time Machine.txt",'r') as f:
  article_text = f.read()

In [10]:
article_text = article_text.lower()

In [11]:
# Demos un vistazo
article_text

'project gutenberg\'s the time machine, by h. g. (herbert george) wells\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  you may copy it, give it away or\nre-use it under the terms of the project gutenberg license included\nwith this ebook or online at www.gutenberg.net\n\n\ntitle: the time machine\n\nauthor: h. g. (herbert george) wells\n\nrelease date: october 2, 2004 [ebook #35]\n[last updated: october 3, 2014]\n\nlanguage: english\n\n\n*** start of this project gutenberg ebook the time machine ***\n\n\n\n\n\n\n\n\n\n\n\n\nthe time machine, by h. g. wells [1898]\n\n\n\n\ni\n\n\nthe time traveller (for so it will be convenient to speak of him)\nwas expounding a recondite matter to us. his grey eyes shone and\ntwinkled, and his usually pale face was flushed and animated. the\nfire burned brightly, and the soft radiance of the incandescent\nlights in the lilies of silver caught the bubbles that flashed and\npassed in our glasses. 

In [12]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 179533


### 2 - Preprocesamiento
- Remover caracteres especiales
- Quitar espacios o saltos

In [13]:
text = re.sub(r'\[[0-9]*\]', ' ', article_text)
text = re.sub(r'\s+', ' ', text)

In [14]:
# Demos un vistazo
text

'project gutenberg\'s the time machine, by h. g. (herbert george) wells this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. you may copy it, give it away or re-use it under the terms of the project gutenberg license included with this ebook or online at www.gutenberg.net title: the time machine author: h. g. (herbert george) wells release date: october 2, 2004 [ebook #35] [last updated: october 3, 2014] language: english *** start of this project gutenberg ebook the time machine *** the time machine, by h. g. wells i the time traveller (for so it will be convenient to speak of him) was expounding a recondite matter to us. his grey eyes shone and twinkled, and his usually pale face was flushed and animated. the fire burned brightly, and the soft radiance of the incandescent lights in the lilies of silver caught the bubbles that flashed and passed in our glasses. our chairs, being his patents, embraced and caressed us rather than submitted 

In [15]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 179132


### 3 - Dividir el texto en sentencias y en palabras

In [16]:
corpus = nltk.sent_tokenize(text)
words = nltk.word_tokenize(text)

In [17]:
# Demos un vistazo
corpus[:10]

["project gutenberg's the time machine, by h. g. (herbert george) wells this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.",
 'you may copy it, give it away or re-use it under the terms of the project gutenberg license included with this ebook or online at www.gutenberg.net title: the time machine author: h. g. (herbert george) wells release date: october 2, 2004 [ebook #35] [last updated: october 3, 2014] language: english *** start of this project gutenberg ebook the time machine *** the time machine, by h. g. wells i the time traveller (for so it will be convenient to speak of him) was expounding a recondite matter to us.',
 'his grey eyes shone and twinkled, and his usually pale face was flushed and animated.',
 'the fire burned brightly, and the soft radiance of the incandescent lights in the lilies of silver caught the bubbles that flashed and passed in our glasses.',
 'our chairs, being his patents, embraced and caressed us rather

In [18]:
# Demos un vistazo
words[:20]

['project',
 'gutenberg',
 "'s",
 'the',
 'time',
 'machine',
 ',',
 'by',
 'h.',
 'g.',
 '(',
 'herbert',
 'george',
 ')',
 'wells',
 'this',
 'ebook',
 'is',
 'for',
 'the']

In [19]:
print("Vocabulario:", len(words))

Vocabulario: 37440


### 4 - Funciones de ayuda para limpiar y procesar el input del usuario
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [20]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula
    # 2 - quitar los simbolos de puntuacion
    # 3 - realiza la tokenización
    # 4 - realiza la lematización
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus de wikipedia

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = "I am sorry, I could not understand you"
    else:
        response = corpus[similar_sentence_number]
    
    corpus.remove(user_input)
    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada. Sugerencias ensayar:
- Grand slam
- tournaments
- nadal
- artificial intelligence

In [22]:
# Se utilizará gradio para ensayar el bot
# Herramienta poderosa para crear interfaces rápidas para ensayar modelos
# https://gradio.app/
import sys
!{sys.executable} -m pip install gradio --quiet

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 206 kB 51.2 MB/s 
     |████████████████████████████████| 1.9 MB 31.6 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 961 kB 48.5 MB/s 
     |████████████████████████████████| 3.0 MB 55.2 MB/s 


In [23]:
import gradio as gr

def bot_response(human_text):
    return generate_response(human_text.lower(), corpus)

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://53888.gradio.app
Interface loading below...


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing

KeyboardInterrupt: ignored